In [2]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision import models, transforms
from PIL import Image


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
img_folder = r"D:\amazon dataset\TRAIN _IMAGES"    # 🔹 change if needed
csv_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv"     

In [11]:

# Load CSV
df = pd.read_csv(csv_path)

# Make sure your CSV has the column that links to image filenames
# For example, 'sample_id' column
img_ids = df["sample_id"].astype(str).tolist()

print(f"Total images found in CSV: {len(img_ids)}")


Total images found in CSV: 75000


In [12]:
model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
model.classifier = torch.nn.Identity()  # remove classification head
model = model.to(device)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [13]:
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [14]:
features = []

with torch.no_grad():
    for img_id in tqdm(img_ids, desc="Extracting CNN features"):
        img_path = os.path.join(img_folder, f"{img_id}.jpg")

        if not os.path.exists(img_path):
            # Missing image — append zeros
            features.append(np.zeros(1536))
            continue

        try:
            img = Image.open(img_path).convert("RGB")
            img_tensor = transform(img).unsqueeze(0).to(device)

            feat = model(img_tensor)
            feat_np = feat.cpu().numpy().flatten()
            features.append(feat_np)
        except Exception as e:
            print(f"Error processing {img_id}: {e}")
            features.append(np.zeros(1536))

features = np.array(features, dtype=np.float32)
print("✅ Extracted features shape:", features.shape)

Extracting CNN features: 100%|██████████| 75000/75000 [1:40:50<00:00, 12.40it/s]  


✅ Extracted features shape: (75000, 1536)


In [15]:
np.save("cnn_features_efficientnetb3_torch.npy", features)
